## Kaggle data management api

In [ ]:
# !pip install kaggle # now download the api token and store it to /home/ec2-user/.kaggle/kaggle.json
# # # !cd /home/ec2-user/SageMaker
# !mkdir /home/ec2-user/.kaggle/
# !mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/kaggle.json
# !chmod 600 /home/ec2-user/.kaggle/kaggle.json # for privacy
# !zip -r -0 -q data.zip /home/ec2-user/SageMaker/data/


### Append the project directory in sys.path for smoother execution

In [11]:
from sys import path

source_proj_dir='/home/ec2-user/SageMaker/Retinopathy2'
# path.append(source_proj_dir)
print(path)

['', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/lib-dynload', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython']


In [12]:
import os
os.chdir("/home/ec2-user/SageMaker/Retinopathy2")
# !kaggle competitions download -c aptos2019-blindness-detection
# !unzip -q /home/ec2-user/SageMaker/aptos2019-blindness-detection.zip -d /home/ec2-user/SageMaker/data1  #-q for quitely no verbose
# mv /home/ec2-user/aptos2019-blindness-detection.zip /home/ec2-user/SageMaker/

In [13]:
# !git clone https://github.com/RamsteinWR/Retinopathy2.git
# !rm -rf Retinopathy2

## Preprocessing the authenication management

In [36]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from tqdm import tqdm
import os 
import urllib.request
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = "dataset-retinopathy"
region_name="us-east-1"
aws_access_key_id = "agduyf"
aws_secret_access_key = "5hagjbfajy"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)    
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print("Upload completed successfully.")
    
def download_dir(s3_folder, local_path, bucket=""):
    """
    params:
    - s3_folder: pattern to match in s3
    - local_path: local_path path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
def download_from_s3(s3_filename, local_path="0.fbx"):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Downloading file {} to {}".format(s3_filename, local_path))
    try:
        s3_client.download_file(bucket, Key=s3_filename, Filename=local_path)
        # it waits till the download completes then moves the execution to forward
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
# download('http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth')
# upload_to_s3("pretrained", 'se_resnext50_32x4d-a260b3a4.pth')
# download_dir(s3_folder='aptos-2015/train_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/train_images_768/', bucket=bucket)
# download_dir(s3_folder='aptos-2015/test_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/test_images_768/', bucket=bucket)
# download_from_s3(s3_filename='pytorch-training-2020-12-29-09-38-13-247/source/sourcedir.tar.gz', local_path="/home/ec2-user/SageMaker/checkpoint/sourcedir.tar.gz")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 44.7 ms, sys: 0 ns, total: 44.7 ms
Wall time: 75 ms


In [ ]:
# !pip install -r requirement.txt

In [40]:
import os
os.chdir('/home/ec2-user/SageMaker/checkpoint/')
!tar -xf /home/ec2-user/SageMaker/checkpoint/sourcedir.tar.gz

<a id='idg4c.1'></a>
## Dependencies
___
### import packages and check SageMaker version

In [16]:
# !pip install -U sagemaker

import json
import torch
import tarfile
import pickle
import matplotlib.pyplot as plt
import torchvision as tv
import pathlib                          # Path management tool (standard library)
import subprocess                       # Runs shell commands via Python (standard library)
import sagemaker                        # SageMaker Python SDK
from sagemaker.pytorch import PyTorch   # PyTorch Estimator for TensorFlow

<a id='idg4c.4'></a>
## Estimator configuration
___

These define the the resources to use for training and how they are configured. Here are some important one to single out:

* **entry_point (str)** – Path (absolute or relative) to the Python source file which should be executed as the entry point to training. If source_dir is specified, then entry_point must point to a file located at the root of source_dir.

* **framework_version (str)** – PyTorch version you want to use for executing your model training code. Defaults to None. Required unless image_uri is provided. List of supported versions: https://github.com/aws/sagemaker-python-sdk#pytorch-sagemaker-estimators.

* **py_version (str)** – Python version you want to use for executing your model training code. One of ‘py2’ or ‘py3’. Defaults to None. Required unless image_uri is provided.

* **source_dir (str)** – Path (absolute, relative or an S3 URI) to a directory with any other training source code dependencies aside from the entry point file (default: None). If source_dir is an S3 URI, it must point to a tar.gz file. Structure within this directory are preserved when training on Amazon SageMaker.

* **dependencies (list[str])** – A list of paths to directories (absolute or relative) with any additional libraries that will be exported to the container (default: []). The library folders will be copied to SageMaker in the same folder where the entrypoint is copied. If ‘git_config’ is provided, ‘dependencies’ should be a list of relative locations to directories with any additional libraries needed in the Git repo.

* **git_config (dict[str, str])** – Git configurations used for cloning files, including repo, branch, commit, 2FA_enabled, username, password and token. The repo field is required. All other fields are optional. repo specifies the Git repository where your training script is stored. If you don’t provide branch, the default value ‘master’ is used. If you don’t provide commit, the latest commit in the specified branch is used.

* **role (str)** – An AWS IAM role (either name or full ARN). The Amazon SageMaker training jobs and APIs that create Amazon SageMaker endpoints use this role to access training data and model artifacts. After the endpoint is created, the inference code might use the IAM role, if it needs to access an AWS resource.

* **instance_count (int)** – Number of Amazon EC2 instances to use for training.

* **instance_type (str)** – Type of EC2 instance to use for training, for example, ‘ml.c4.xlarge’.

* **volume_size (int)** – Size in GB of the EBS volume to use for storing input data during training (default: 30). Must be large enough to store training data if File Mode is used (which is the default).

* **model_uri (str)** – URI where a pre-trained model is stored, either locally or in S3 (default: None). If specified, the estimator will create a channel pointing to the model so the training job can download it. This model can be a ‘model.tar.gz’ from a previous training job, or other artifacts coming from a different source. In local mode, this should point to the path in which the model is located and not the file itself, as local Docker containers will try to mount the URI as a volume.

* **output_path (str)** - S3 location for saving the training result (model artifacts and output files). If not specified, results are stored to a default bucket. If the bucket with the specific name does not exist, the estimator creates the bucket during the fit() method execution. file:// urls are used for local mode. For example: ‘file://model/’ will save to the model folder in the current directory.

### Define the hyperparamters for EC2 training

In [17]:
hyperparameters = {
    "seed": 42,
    #   "fast": false,
    #   "mixup": false,
    #   "balance": false,
    #   "balance_datasets": false,
    #   "swa": false,
    #   "show": false,
    #   "use_idrid": false,
    #   "use_messidor": false,
    #   "use_aptos2015": false,
    # "use_aptos2019": "",
    # "verbose": "",
    #   "coarse": false,
    "accumulation-steps": 1,
    "data-dir": "/opt/ml/input/data",
    "model": "seresnext50d_gwap",
    "batch-size": 4,
    "epochs": 100,
    "early-stopping": 10,
    # "fold": [
    #     0,
    #     1,
    #     2,
    #     3
    # ],
    #   "freeze_encoder": false,
    "learning-rate": 0.0001,
    # "criterion_reg": [
    #     "mse"
    # ],
    #   "criterion_ord": null,
    # "criterion_cls": [
    #     "focal_kappa"
    # ],
    "l1": 0.0002,
    # "l2": 0,
    "optimizer": "AdamW",
    #   "preprocessing": null,
    #   "checkpoint": null,
      "workers": 8,
    "augmentations": "medium",
    #   "tta": null,
#     "transfer": "pretrained",
    #   "fp16": true,
    "scheduler": "multistep",
    "size": 1024,
    "weight-decay": 0.0001,
    #   "weight_decay_step": null,
#     "dropout": 0.2,
    # "warmup": 0,
    #   "experiment": null
}


### Define the estimator configuration for EC2 training

In [18]:
estimator_config = {
    'entry_point': 'train_reg_1.py',
    'source_dir': source_proj_dir,
    'framework_version': '1.6.0',
    'py_version': 'py3',
    'instance_count': 1,
    'instance_type': 'ml.p3.16xlarge',
    'volume_size':200,
    'role': sagemaker.get_execution_role(),
    'output_path': f's3://{bucket}/training_job',
    'checkpoint_local_path': None,
    'max_run':86400*3,
    'hyperparameters': hyperparameters,
}

### Create the estimator configured for EC2 training

In [19]:
pytorch_estimator = PyTorch(**estimator_config)

### Define the data channels using the proper S3 URIs

In [20]:
data_channels = {
    'aptos-2015': f's3://{bucket}/aptos-2015',
    'aptos-2019':   f's3://{bucket}/aptos-2019',
    'idrid':   f's3://{bucket}/idrid',
    'messidor':   f's3://{bucket}/messidor',
    'origa':   f's3://{bucket}/origa',
    'stare':   f's3://{bucket}/stare',
#     'pretrained':   f's3://{bucket}/pretrained', # already loading the pretrained after downloading 
}

In [21]:
# pytorch_estimator.fit(data_channels)

# SageMaker endpoint

To deploy the model you previously trained, you need to create a Sagemaker Endpoint. This is a hosted prediction service that you can use to perform inference.

## Finding the model

This notebook uses a stored model if it exists. If you recently ran a training example that use the `%store%` magic, it will be restored in the next cell.

Otherwise, you can pass the URI to the model file (a .tar.gz file) in the `model_data` variable.

You can find your model files through the [SageMaker console](https://console.aws.amazon.com/sagemaker/home) by choosing **Training > Training jobs** in the left navigation pane. Find your recent training job, choose it, and then look for the `s3://` link in the **Output** pane. Uncomment the model_data line in the next cell that manually sets the model's URI.

In [24]:
# Retrieve a saved model from a previous notebook run's stored variable
# %store -r model_data
os.chdir("/home/ec2-user/SageMaker/Retinopathy2")

# If no model was found, set it manually here.
model_data = 's3://dataset-retinopathy/pytorch-training-2020-12-29-09-38-13-247/source/sourcedir.tar.gz'

print("Using this model: {}".format(model_data))

Using this model: s3://dataset-retinopathy/pytorch-training-2020-12-29-09-38-13-247/source/sourcedir.tar.gz


## Create a model object

You define the model object by using SageMaker SDK's `PyTorchModel` and pass in the model from the `estimator` and the `entry_point`. The endpoint's entry point for inference is defined by `model_fn` as seen in the following code block that prints out `inference.py`. The function loads the model and sets it to use a GPU, if available.

### Load the weights back into a PyTorch model
Since the model was trained on a GPU we need to use the `map_location=torch.device('cpu')` kwarg to load the model on a CPU backed notebook instance.

In [25]:
import sagemaker
role = sagemaker.get_execution_role()

from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(model_data=model_data, source_dir='code',
                        entry_point='inference.py', role=role, framework_version='1.6.0', py_version='py3')

### Deploy the model on an endpoint

You create a `predictor` by using the `model.deploy` function. You can optionally change both the instance count and instance type.

In [26]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

FileNotFoundError: [Errno 2] No such file or directory: 'code'

## Test the model
You can test the depolyed model using samples from the test set.


In [ ]:
# Download the test set
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_set = datasets.MNIST('data', download=True, train=False, 
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                            ]))


# Randomly sample 16 images from the test set
test_loader = DataLoader(test_set, shuffle=True, batch_size=16)
test_images, _ = iter(test_loader).next()

# inspect the images
import torchvision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    img = img.numpy()
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img)
    return

# unnormalize the test images for displaying
unnorm_images = (test_images * 0.3081) + 0.1307

print("Sampled test images: ")
imshow(torchvision.utils.make_grid(unnorm_images))



In [ ]:
# Send the sampled images to endpoint for inference
outputs = predictor.predict(test_images.numpy())
predicted = np.argmax(outputs, axis=1)

print("Predictions: ")
print(predicted.tolist())

## Cleanup

If you don't intend on trying out inference or to do anything else with the endpoint, you should delete it.

In [ ]:
predictor.delete_endpoint()